In [1]:
!pip install transformers gradio torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 801.0 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.7 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 18.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 17.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.7/141.7 kB 17.5 M

In [5]:
!pip install --upgrade gradio

In [7]:
# Cell 1: Import required libraries and set up the model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# Load pre-trained model and tokenizer
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Cell 2: Create a simple knowledge base
knowledge_base = {
    "depression": "Depression is a common mental health disorder characterized by persistent feelings of sadness and loss of interest. If you're experiencing symptoms of depression, it's important to reach out to a mental health professional for support.",
    "anxiety": "Anxiety is a normal reaction to stress, but when it becomes excessive, it may be classified as an anxiety disorder. Techniques like deep breathing, mindfulness, and cognitive-behavioral therapy can help manage anxiety.",
    "help": "If you're in crisis, please reach out to a mental health professional or call a crisis hotline immediately. Remember, seeking help is a sign of strength, not weakness.",
}

# Cell 3: Define the chatbot response function
def chatbot_response(input_text):
    # Check if the input matches any key in our knowledge base
    for key in knowledge_base:
        if key in input_text.lower():
            return knowledge_base[key]
    
    # If no match in knowledge base, use the language model
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors="pt")
    chat_history_ids = model.generate(
        input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature=0.8
    )
    response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Cell 4: Add basic error handling and safety features
def safe_chatbot_response(input_text):
    try:
        if any(word in input_text.lower() for word in ["suicide", "kill myself", "end my life"]):
            return "I'm concerned about what you've said. If you're having thoughts of suicide, please call a suicide prevention hotline immediately. Your life matters."
        return chatbot_response(input_text)
    except Exception as e:
        return f"I'm sorry, but I encountered an error. Please try again or seek help from a mental health professional. Error: {str(e)}"

# Cell 5: Create and launch the Gradio interface
iface = gr.Interface(
    fn=safe_chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Type your message here..."),
    outputs="text",
    title="Mental Health Support Chatbot",
    description="This is a basic chatbot for mental health support. Please note that this is not a substitute for professional help."
)

iface.launch()

# Cell 6 (Optional): Test the chatbot with some sample inputs
print(safe_chatbot_response("I've been feeling really down lately."))
print(safe_chatbot_response("What are some ways to manage anxiety?"))
print(safe_chatbot_response("I need help."))
print(safe_chatbot_response("Tell me a joke."))

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Why are you doing this to yourself?
Anxiety is a normal reaction to stress, but when it becomes excessive, it may be classified as an anxiety disorder. Techniques like deep breathing, mindfulness, and cognitive-behavioral therapy can help manage anxiety.
If you're in crisis, please reach out to a mental health professional or call a crisis hotline immediately. Remember, seeking help is a sign of strength, not weakness.
What is Aleppo?
